In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 403.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 47.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "drive/MyDrive/project_cs685/model/llama-2-finetuned/causal-reasoning-finetuned/"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_llama2_ft_response(prompt):
  input_text = prompt
  input = tokenizer(input_text, return_tensors='pt').to(device)

  output = model.generate(input['input_ids'], \
                        attention_mask=input["attention_mask"], \
                        #pad_token_id=pad_token_id, \
                        output_scores=True, \
                        return_dict_in_generate=True, \
                        do_sample=False, temperature=None, top_p=None, \
                        max_length=500)

  len_output = len(output['scores'])
  output_tokens = output['sequences'].squeeze()[-len_output:]
  output_text = tokenizer.decode(output['sequences'].squeeze(), skip_special_tokens=True)


  topk_values, topk_indices, topk_texts = None, None, None
  for i in range(len_output):
    _token_text = tokenizer.decode(output_tokens[i], skip_special_tokens=True)
    if ("yes" in _token_text.lower()) or ("no" in _token_text.lower()):

      # logits and log probabilities
      logits = output['scores'][i]
      log_probs = torch.log(torch.softmax(logits, dim=-1))

      n = 2
      topk_values, topk_indices = torch.topk(log_probs, k=n)
      topk_texts = [tokenizer.decode(topk_indices.squeeze()[i]) for i in range(n)]
      break

  return {
      'generated_text': output_text,
      'topk_log_probs': topk_values.squeeze().tolist(),
      'topk_indices': topk_indices.squeeze().tolist(),
      'topk_texts': topk_texts
  }


In [ ]:
prompt = """<s>[INST] Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Refer to the given context to respond the question with either 'yes' or 'no'.

Input:
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Glopp has a direct effect on fritz and zibbo. Fritz has a direct effect on flump. Flump has a direct effect on zibbo. Glopp is unobserved. The overall probability of fritz is 10%. For those who are not fritz, the probability of zibbo is 30%. For those who are fritz, the probability of zibbo is 48%. Is zibbo less likely than not zibbo overall?

### Response:
 [/INST] """

In [ ]:
%%time
get_llama2_ft_response(prompt)

CPU times: user 137 ms, sys: 3.14 ms, total: 140 ms
Wall time: 139 ms


{'generated_text': "[INST] Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nRefer to the given context to respond the question with either 'yes' or 'no'.\n\nInput:\nImagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Glopp has a direct effect on fritz and zibbo. Fritz has a direct effect on flump. Flump has a direct effect on zibbo. Glopp is unobserved. The overall probability of fritz is 10%. For those who are not fritz, the probability of zibbo is 30%. For those who are fritz, the probability of zibbo is 48%. Is zibbo less likely than not zibbo overall?\n\n### Response:\n [/INST]  No.",
 'topk_log_probs': [-0.21740950644016266, -2.295534372329712],
 'topk_indices': [1939, 694],
 'topk_texts': ['No', 'no']}

In [ ]:
# The instruction dataset to use
dataset_name = "drive/MyDrive/project_cs685/dataset/test_dataset.csv"

In [ ]:
test_dataset_df = pd.read_csv(dataset_name)

In [ ]:
test_dataset_df.head()

,question_id,desc_id,given_info,question,answer,meta,reasoning,background
0,3888,nonsense13-frontdoor-marginal-modelNone-spec5-q1,The overall probability of fritz is 10%. For t...,Is zibbo less likely than not zibbo overall?,yes,"{'story_id': 'nonsense13', 'graph_id': 'frontd...",{'step0': 'Let V1 = glopp; X = fritz; V3 = flu...,"Imagine a self-contained, hypothetical world w..."
1,8401,nonsense17-frontdoor-nie-modelNone-spec8-q0,"For those who are not bleen, the probability o...",Does bleen positively affect zibby through flarn?,no,"{'story_id': 'nonsense17', 'graph_id': 'frontd...",{'step0': 'Let V1 = quop; X = bleen; V3 = flar...,"Imagine a self-contained, hypothetical world w..."
2,8132,nonsense17-confounding-ate-modelNone-spec7-q1,"For those who are not flarn and are not bleen,...",Will bleen decrease the chance of zibby?,no,"{'story_id': 'nonsense17', 'graph_id': 'confou...",{'step0': 'Let V1 = flarn; X = bleen; Y = zibb...,"Imagine a self-contained, hypothetical world w..."
3,4701,nonsense14-confounding-ate-modelNone-spec0-q0,For those who are not flurnt and are not plizz...,Will plizz increase the chance of brifft?,no,"{'story_id': 'nonsense14', 'graph_id': 'confou...",{'step0': 'Let V1 = flurnt; X = plizz; Y = bri...,"Imagine a self-contained, hypothetical world w..."
4,7168,nonsense16-mediation-ate-modelNone-spec9-q1,"For those who are not gruntz, the probability ...",Will gruntz decrease the chance of flurrk?,no,"{'story_id': 'nonsense16', 'graph_id': 'mediat...",{'step0': 'Let X = gruntz; V2 = zowpxo; Y = fl...,"Imagine a self-contained, hypothetical world w..."


In [ ]:
test_dataset_df.columns

Index(['question_id', 'desc_id', 'given_info', 'question', 'answer', 'meta',
       'reasoning', 'background'],
      dtype='object')

In [ ]:
test_dataset_df2 = test_dataset_df[['question_id', 'desc_id', 'given_info', 'question', 'answer', 'background']].reset_index(drop = True)

In [ ]:
instruction_text = "Refer to the given context to respond the question with either 'yes' or 'no'."

#"If you're unsure of the answer, simply state 'na' without guessing."
#"Use the following context to answer the question. The answer will be 'yes' or 'no'. If you don't know the answer, just say 'na', don't try to make up an answer."


def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{instruction_text}"
    input_context = f"{INPUT_KEY}\n{sample['background']} {sample['given_info']} {sample['question']}" #if sample["input"] else None
    response = f"{RESPONSE_KEY}\n" # "{sample['answer']}"
    #end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response] if part] # response, end

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)
    formatted_prompt = f"<s>[INST] {formatted_prompt} [/INST]"

    # Store the formatted prompt template in a new key "text"
    #sample["text"] = formatted_prompt

    return formatted_prompt #sample


In [ ]:
test_dataset_df2["prompt_text"] = test_dataset_df2.apply(create_prompt_formats, axis = 1)

In [ ]:
test_dataset_df2["llama2_ft_response"] = test_dataset_df2.prompt_text.apply(get_llama2_ft_response)

In [ ]:
test_dataset_df2["llama2_ft_generated_text"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['generated_text'])

test_dataset_df2["top1_log_probs"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_log_probs'][0])
#test_dataset_df2["top1_indices"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_indices'][0])
test_dataset_df2["top1_texts"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_texts'][0])

test_dataset_df2["top2_log_probs"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_log_probs'][1])
#test_dataset_df2["top2_indices"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_indices'][1])
test_dataset_df2["top2_texts"] = test_dataset_df2.llama2_ft_response.apply(lambda x: x['topk_texts'][1])

In [ ]:
test_dataset_df2["llama2_ft_generated_text"][2]

"[INST] Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nRefer to the given context to respond the question with either 'yes' or 'no'.\n\nInput:\nImagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Flarn has a direct effect on bleen and zibby. Bleen has a direct effect on zibby. For those who are not flarn and are not bleen, the probability of zibby is 21%. For those who are not flarn and are bleen, the probability of zibby is 27%. For those who are flarn and are not bleen, the probability of zibby is 20%. For those who are flarn and are bleen, the probability of zibby is 53%. The overall probability of flarn is 37%. Will bleen decrease the chance of zibby?\n\n### Response:\n [/INST]  No."

In [ ]:
def process_llama2_ft_response(response_text):
  _index = response_text.find("[/INST]")
  if _index != -1:
    response_text = response_text[(_index + len("[/INST]")):]
  else:
    return 'na'
  response_text_lst = response_text.split(' ')
  if len(response_text_lst) > 0:
    for i in response_text_lst:
      if 'yes' in i.lower():
        return 'yes'
      elif 'no' in i.lower():
        return 'no'
  else:
    return 'na'

In [ ]:
test_dataset_df2["llama2_ft_response_processed"] = test_dataset_df2.llama2_ft_generated_text.apply(process_llama2_ft_response)

In [ ]:
test_dataset_df2.head()

,question_id,desc_id,given_info,question,answer,background,prompt_text,llama2_ft_response,llama2_ft_generated_text,top1_log_probs,top1_texts,top2_log_probs,top2_texts,llama2_ft_response_processed
0,3888,nonsense13-frontdoor-marginal-modelNone-spec5-q1,The overall probability of fritz is 10%. For t...,Is zibbo less likely than not zibbo overall?,yes,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,{'generated_text': '[INST] Below is an instruc...,[INST] Below is an instruction that describes ...,-0.215682,No,-2.309432,no,no
1,8401,nonsense17-frontdoor-nie-modelNone-spec8-q0,"For those who are not bleen, the probability o...",Does bleen positively affect zibby through flarn?,no,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,{'generated_text': '[INST] Below is an instruc...,[INST] Below is an instruction that describes ...,-0.637751,no,-1.419001,No,no
2,8132,nonsense17-confounding-ate-modelNone-spec7-q1,"For those who are not flarn and are not bleen,...",Will bleen decrease the chance of zibby?,no,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,{'generated_text': '[INST] Below is an instruc...,[INST] Below is an instruction that describes ...,-0.100665,No,-2.788165,no,no
3,4701,nonsense14-confounding-ate-modelNone-spec0-q0,For those who are not flurnt and are not plizz...,Will plizz increase the chance of brifft?,no,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,{'generated_text': '[INST] Below is an instruc...,[INST] Below is an instruction that describes ...,-0.489506,No,-1.677006,Yes,no
4,7168,nonsense16-mediation-ate-modelNone-spec9-q1,"For those who are not gruntz, the probability ...",Will gruntz decrease the chance of flurrk?,no,"Imagine a self-contained, hypothetical world w...",<s>[INST] Below is an instruction that describ...,{'generated_text': '[INST] Below is an instruc...,[INST] Below is an instruction that describes ...,-0.023237,No,-4.179487,no,no


In [ ]:
test_dataset_df2.to_csv('results/output_data_llama2_finetuned.csv', header = True, index = False)